# Upload Downloaded AML Model Files To Azure Blobs

In this notebook, you will download a model from azure machine learning workspace to your local envrionment, then upload the model files as azure storage blobs.

## Prerequisites
*  Azure Machine Learning Workspace
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [AZML-SDK-INSTALL](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`.
*  An registered AML machine learning model. For how to train and register model, please see [tf2-cifar10-train](distributed-tf2-cifar10/distributed-tf2-cifar10.ipynb)
*  Azure storage blob client library. For more details, please see [here](https://docs.microsoft.com/en-us/python/api/overview/azure/storage-blob-readme?view=azure-python). 

In [ ]:
from azureml.core import Workspace
import os

## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Download model from AML Workspace

In [ ]:
model_path = 'cifar10model'
model_name = "slcifar10"

ws.models[model_name].download(target_dir=model_path, exist_ok=True)

## Upload model files to Azure storage blobs

In [ ]:
from azure.storage.blob import BlobServiceClient
    
connection_string = "<storage account connection string>"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    
container_name = "tfmodel"
container_client = blob_service_client.get_container_client(container_name)

for root, dirs, files in os.walk(model_path):
    for file in files:
        source_file = os.path.join(root, file)
        blob_name = source_file
        blob_client = container_client.get_blob_client(blob_name)
        with open(source_file, "rb") as data:
            blob_client.upload_blob(data, blob_type="BlockBlob")

Now you should see the files are uploaded to Blob Storage